ibss마감결과의 오류를 검색 (수수료 정산작업전에 사용)
--- goms와 비교/ 월납오류 검출

In [20]:
import pandas as pd
import GladUtil as gd
pd.set_option('display.max_rows', 1000)
대상 = ['장기','생보','일반','자동차']
미유지 = ['실효','해지','철회','취소']

path_ibss ='e:/글로벌백업(180329)/04.수수료작업/202211수수료/'
ibss_file = []
ibss_file.append('마감실적리스트(장기)-202212270600.xlsx')
ibss_file.append('마감실적리스트(생보)-202212270600.xlsx')
ibss_file.append('마감실적리스트(일반)-202212270600.xlsx')
ibss_file.append('마감실적리스트(자동차)-202212270600.xlsx')
goms_file = 'goms.xlsx'

gb = '생보'
date_cond = '2022-11-01'
 
in_file = ibss_file[대상.index(gb)]
df = pd.read_excel(path_ibss+in_file, header=0, dtype={'증권번호':object})
df = df[df['본부']=='GLAD']

goms_file = 'goms.xlsx'
df2 = pd.read_excel(path_ibss+goms_file,header=0, dtype={'증권번호':object})

In [21]:
if gb in ('일반','자동차'):
    df['보종'] = gb        
    if gb == '자동차':
        df.rename(columns={'개시일':'계약일','보험료':'월납보험료','납입':'구분'},inplace=True)
        df2['상품종류'] = gb
    else:
        df.rename(columns={'납입보험료':'월납보험료'},inplace=True)
    
''' ibss - 당월건'''
df_ibss = df[['보험회사','증권번호','담당ID','담당','보종','구분','계약일','월납보험료']].copy()
df_ibss = df_ibss[df_ibss['계약일'] >= date_cond]

''' goms - 해당 보종'''
df_goms = df2[df2['계약종류'] == gb][['보험사','증권번호','모집사원번호','수금사원명','계약종류','상품종류','계약상태','계약일자','초회보험료']].copy()

''' join 및 값 비교'''
df_ply = pd.merge(df_ibss, df_goms, how = 'outer',
            on=['증권번호'],indicator = True).reset_index().astype({"_merge": str}).fillna(0)
df_ply.drop(columns='index',inplace=True)
DICT_CONV = {'하나손해보험':'하나손보','AIG손해보험':'AIG손보','미래에셋생명':'미래에셋','메트라이프생명':'메트라이프','신한라이프':'신한생명'}
# df_ply['보험사'] = df_ply['보험사'].replace({'하나손해보험':'하나손보','AIG손해보험':'AIG손보','미래에셋생명':'미래에셋'})
df_ply['보험사'] = df_ply['보험사'].apply(lambda x: DICT_CONV.get(x,x))

df_ply['사원비교']= ["일치" if str(x)[:2] == str(y)[:2]  else "불일치" 
                 for x,y in zip(df_ply['담당'],df_ply['수금사원명'])]

df_ply['사원ID비교']= ["일치" if (str(x) == str(y) or str(x)[:2] =='ba') else "불일치" 
                 for x,y in zip(df_ply['담당ID'],df_ply['모집사원번호'])]

df_ply['보종비교']= ["일치" if ( str(x) in str(y) or str(y) in str(x) )  else "불일치" 
                 for x,y in zip(df_ply['보종'],df_ply['상품종류'])]

df_ply['증번비교']= ["일치" if x == 'both'  else ( 'ibss만 존재' if x == 'left_only' else "goms만 존재") 
                 for x in df_ply['_merge'] ]

df_ply['회사비교']= ["일치" if str(x) == str(y)  else ("불일치" if str(z)=='both' else "")
                 for x,y,z in zip(df_ply['보험회사'],df_ply['보험사'],df_ply['_merge'])]

exp = f"(구분 in {미유지}) and 월납보험료 == 0 "
df_temp = df.query(exp)[['보험회사','증권번호','담당','보종','구분','계약일','월납보험료']]
df_temp['보험료비교'] = '월납오류'

exp  = f" ( 사원비교 != '일치') or ( 사원ID비교 != '일치') or (보종비교 != '일치') or (증번비교 != '일치') or (회사비교 != '일치') "
df_error = pd.concat([df_ply.query(exp),df_temp])

df_error.fillna('').to_excel(f'd:/temp/ibss_마감오류_{gb}.xlsx',index = False)
print('end')

end


년차별 환산보험료 다른건 추출

In [22]:
''' 021.4.1이후 신계약대상으로 년차별 환산보험료를 IBSS와 GOMS를 비교한다'''
path_ibss ='e:/글로벌백업(180329)/04.수수료작업/'
ym = '202211'
goms_file = 'goms.xlsx'
gb = '생보'

''' ibss - 당월건''' 
in_file = ibss_file[대상.index(gb)]
df = pd.read_excel(f'{path_ibss}{ym}수수료/{in_file}', header=0, dtype={'증권번호':object})
df = df[df['본부']=='GLAD']

# df_ibss = df[['보험회사','증권번호','담당','보종','구분','계약일','납입회차','월납보험료','환산보험료']].copy()
df_ibss = df[['보험회사','증권번호','담당','보종','구분','계약일','납입회차','월납보험료','1차년환산보험료','2차년환산보험료','3차년환산보험료']].copy()
date_cond = df_ibss['계약일'] >= '2021-04-01'
# seq_cond = df_ibss['납입회차'].isin([13,25])
seq_cond = ( (df_ibss['납입회차'] >=13) & (df_ibss['납입회차'] <=36) )
df_ibss = df_ibss[date_cond & seq_cond]
# 납입회차에 맞는 년차별 환산보험료 컬럼 생성
df_ibss['차년'] = df_ibss.납입회차.apply(lambda x: str(min(int(x // 12.1)+1, 3))+'차년환산보험료')
df_ibss['환산보험료'] = df_ibss.apply(lambda x: x[x['차년']], axis=1)


# goms파일 read
ym_list = df_ibss['계약일'].str.replace('-','').str[:6].unique()
df_goms = pd.DataFrame()
g_cols = ['보험사','증권번호','상품종류','계약상태','초회보험료','1차년성적','2차년성적','3차년성적']
for i in ym_list:
    df2 = pd.read_excel(f'{path_ibss}{i}수수료/{goms_file}',header=0, dtype={'증권번호':object})
    df2 = df2[df2['계약종류'] == gb][g_cols].copy()
    df2['증권번호'] = [ '00'+ y if x.startswith('신한') and len(y)< 11 else y for x,y in zip(df2['보험사'],df2['증권번호']) ]
    df_goms = pd.concat([df_goms,df2])

''' join 및 값 비교'''

df_ply = pd.merge(df_ibss, df_goms, how = 'left',
            on=['증권번호'],indicator = True).reset_index().astype({"_merge": str}).fillna(0)
df_ply.drop(columns='index',inplace=True)
# dict_s = {13:'2차년성적',25:'3차년성적'}  #환산컬럼
# df_ply['goms환산'] = df_ply.apply(lambda x: x[dict_s.get(x['납입회차'])],axis=1)
# 납입회차에 맞는 년차별 환산보험료 컬럼 생성
df_ply['차년'] = df_ply.납입회차.apply(lambda x: str(min(int(x // 12.1)+1, 3))+'차년성적')
df_ply['goms환산'] = df_ply.apply(lambda x: x[x['차년']], axis=1)

exp  = f"( ( 환산보험료 != goms환산 and goms환산 !=0 ) or _merge != 'both' )  "
df_error2 = df_ply.query(exp)

df_error2.fillna('').to_excel(f'd:/temp/ibss_환산오류_{gb}.xlsx',index = False)
print('end of job')

end of job
